
__Implementing semantic search within a document using word embeddings__  
https://towardsdatascience.com/beyond-ctrl-f-44f4bec892e9  
https://github.com/fabio-a-oliveira/semantic-search/blob/main/semantic_search.ipynb

---

In this notebook, we'll apply _Natural Language Processing (NLP)_ techniques to implement a semantic search within a document.  
This means that, instead of searching for a literal word or sequence of words (as you would when you use `CTRL+F` in Notepad, Microsoft Word etc), we'll be searching for terms, sentences or paragraphs of ___similar meaning___. If you ever had to search a massive document for a piece of information that you cannot recall exactly, you will agree that this application can be extremely useful and time-saving.

In summary, this is what we are going to do:

1. Use ___word embeddings___ to convert every word in the book and the requested sentence to a dense vector representation (in this case, we'll use GloVe embeddings);
2. Apply a ___part-of-speech___ (POS) mask to both the book and the requested sentence: every word that does not belong to a list of relevant _parts-of-speech_ will have the embedding converted to a null vector;
3. Apply a ___bag-of-words___ approach to sentence embedding: average the embeddings of every word in the sentence and get a single vector to represent its semantic content;
4. Apply the _bag-of-words_ sentence embedding and POS filter to the entire book by using a ___sliding window___ with the length of the requested sentence plus a selected margin and averaging the embeddings of words within the window;
5. Calculate the ___cosine distance___ between the requested sentence embedding and the sliding window embeddings
6. Select the position in the book with the shortest distance to the requested sentence.

To illustrate the concept, we'll download the .txt file of the book _Pride and Prejudice_ from the Project Gutenberg website and we'll show two applications of the technique:

1. We will take a sentence from the book and search the text for several increasingly altered versions of it;
2. We will take several excerpts from the Brazilian Portuguese version of the book, translate them back to English (which results in sentences with equivalent meaning but significantly different wording), and find the correspoding match in the original version.

---

## Setup

### Install and import libraries

- Most of the NLP resources we need are available in the `nltk` (Natural Language Toolkit) package.

In [ ]:
import numpy as np
import requests, nltk

from os import mkdir, getcwd, listdir
from os.path import join
from zipfile import ZipFile

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

HOME = getcwd()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/catsmile/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/catsmile/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Download and prepare word embeddings and Project Gutenberg catalogue

The heart and soul of capturing word meaning will be provided by GloVe word embeddings.   
In order to use them, we download the embeddings from the Stanford NLP website and 

In [ ]:
if 'data' not in listdir(): mkdir('data')

if 'glove.6B.zip' not in listdir(join(HOME, 'data')):
    URL_GloVe = 'http://nlp.stanford.edu/data/glove.6B.zip'
    r = requests.get(URL_GloVe).content
    with open(join('data', 'glove.6B.zip'), 'wb') as file:
        file.write(r)

if 'glove.6B.300d.txt' not in listdir(join(HOME, 'GldataoVe')):
    z = ZipFile(join(HOME, 'data','glove.6B.zip'))
    z.extractall(join(HOME, 'data'))

Convert the raw data to a Python dictionary.

In [ ]:
embedding_dict = {}

with open(join(HOME, 'data', 'glove.6B.300d.txt')) as file:
    for line in file:
        word, vector = line.split(' ', maxsplit=1)
        vector = np.array(vector.split(' ')).astype('float')
        embedding_dict.update({word:vector})

### Helper functions

In [ ]:
def sequence_embedding(content, embedding_dict, 
                       allowed_pos = ['NN','NNS','NNP','NNPS','JJ','RB','VB',
                                      'VBG','VBN','VBP','VBZ','VBD']):
    """Takes text content as a string and returns a matrix with the embeddings for each word according to a given dictionary.
        Use a list of allowed parts-of-speech, to give more control over what categories of words will be kept or masked out.
        This list, also known as list of ___stop words___ - frequently occurrying words that can be removed from the bag-of-words representation without much loss of meaning.

    Args:
        content (str): String to be embedded
        embedding_dict (dictionary): Embedding dictionary
        allowed_pos (list, optional): List of allowed parts-of-speech. Defaults to ['NN','NNS','NNP','NNPS','JJ','RB','VB', 'VBG','VBN','VBP','VBZ','VBD'].

    Returns:
        np.array: The embedded `content`.
    """

    # basic cleanup
    content = content.replace('\n', ' ').replace('_', "").lower()

    # tokenize content
    tokens = nltk.word_tokenize(content)

    # get mask indicating tokens that are in or out of allowed parts-of-speech
    pos_mask = np.array([int(tag[1] in allowed_pos) 
                         for tag in nltk.pos_tag(tokens)]).reshape((-1,1))

    # get embedding and apply mask
    embedding_dim = len(list(embedding_dict.values())[0])
    embedding = np.array([embedding_dict[token] 
                          if token in embedding_dict.keys() 
                          else np.zeros(embedding_dim) 
                          for token in tokens])
    embedding *= pos_mask

    return embedding

def cosine_distance(vec1, vec2):
    """Calculate the _cosine distance_ between embeddings.
        Standalone function from the one in `scipy` package

    Args:
        vec1 (np.array): Embedding 1
        vec2 (np.array): Embedding 2

    Returns:
        int: Distance between Embedding 1 and 2
    """

    dot_prod = np.dot(vec1, vec2)
    norm1 = np.sqrt(np.sum(vec1 ** 2))
    norm2 = np.sqrt(np.sum(vec2 ** 2))

    if norm1 == 0 or norm2 == 0:
        return 1
    else:
        return 1 - dot_prod / (norm1 * norm2)
    
def sliding_distance(book, excerpt, margin = 0):
    """Calculates the cosine distance between 
        - the input exerpt (embedded exerpt)
        - each possible sentence in the book (the embeddings of a sliding window of words throughout the entire book)

    Args:
        book (np.array): The embedded book
        excerpt (np.array): The embedded input excerpt
        margin (int, optional): To search for sentences that are longer than the provided excerpt. Defaults to 0.

    Returns:
        ndarray: The array
    """

    # The length of the sliding window corresponds to the length of the given excerpt plus a selected margin.  
    mvg_avg_widgth = excerpt.shape[0] + margin

    # bag-of-words excerpt embedding
    excerpt_embedding = excerpt.mean(axis=0)

    # moving average of the book embedding, 
    # considering the length of the excerpt + a margin
    mvg_avg_embedding = np.array([book[line-mvg_avg_widgth:line].mean(axis=0) 
                            for line in range(mvg_avg_widgth, book.shape[0])])

    # sliding distance: distance between sliding window and excerpt embeddings
    distance = np.array([cosine_distance(mvg_avg_embedding[line,:], 
                                         excerpt_embedding) 
                         for line in range(mvg_avg_embedding.shape[0])])

    return distance

def find_match(reference, match_position, match_length):
    """Takes a chosen `match_position` and sentence `match_length` and returns the corresponding excerpt from the `reference`.  

    Args:
        reference (str): The document in raw text.
        match_position (int): tokenized position to be extracted
        match_length (int): number of tokens to extract starting from `match_position`

    Returns:
        _type_: _description_
    """

    # basic cleanup
    reference = reference.replace('\n', ' ').replace('_', "").lower()

    match = nltk.word_tokenize(reference)
    match = match[match_position : match_position + match_length]
    match = ' '.join(match).replace(' ,',',').replace(' .','.')
    return "["+str(match_position)+"] "+match


### Main function

In [ ]:
def locate_excerpt(excerpt, book, margin = 0, count = 1):
    """_summary_

    Args:
        excerpt (str): The input exerpt to be searched.
        book (str): The document in raw text.
        margin (int, optional): To search for sentences that are longer than the provided excerpt. Defaults to 0.
        count (int, optional): Number of excerpts to search for. Defaults to 1.

    Returns:
        list: Excerpts found from the book
    """

    # embed excerpt and get word count
    book_embedding = sequence_embedding(book, embedding_dict)
    excerpt_embedding = sequence_embedding(excerpt, embedding_dict)
    excerpt_word_count = len(nltk.word_tokenize(excerpt))

    # calculate distances
    distances = sliding_distance(book_embedding, excerpt_embedding, margin = margin)

    # find match and print it
    # find_match(book, distances.argmin(), excerpt_word_count + margin)
    
    # `ndarray.argmin()` method to find the position with the smallest distance.
    # `ndarray.argsort()` method to get the top distances starting from the smallest
    return [ find_match(book, distances.argsort()[i], excerpt_word_count + 2*margin) 
                for i in range(count) ]

### Additional function

In [ ]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster

def merge_overlapping_results(data_sentences, similarity_threshold=0.4):
    """From a list of sentences, group them into clusters based on their similarity

    Args:
        data_sentences (list): List of sentences
        similarity_threshold (float, optional): Threshold to group sentences. Lower the stricter. Defaults to 0.4.

    Returns:
        Dictionary: A dictionary of clusters and their sentences.
    """

    # Load the spaCy English model
    nlp = spacy.load("en_core_web_sm")

    # Calculate sentence embeddings using spaCy
    data_embeddings = [nlp(sentence).vector for sentence in data_sentences]

    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(data_embeddings)

    # Perform hierarchical clustering
    linkage_matrix = linkage(similarity_matrix, method='complete')

    # Apply clustering with a similarity threshold
    cluster_labels = fcluster(linkage_matrix, t=similarity_threshold, criterion='distance')

    # Initialize a dictionary to store merged results for each cluster
    merged_results = {}

    # Group sentences by cluster label
    for idx, cluster_label in enumerate(cluster_labels):
        if cluster_label not in merged_results:
            merged_results[cluster_label] = [data_sentences[idx]]
        else:
            merged_results[cluster_label].append(data_sentences[idx])

    return list(merged_results.values())

---
## Application

__First application__  
Take a sentence from the book and search the document for several modified versions of it.

__Second application__  
Take several sentences from the Brazilian Portuguese translation of the book, translate them back to English and search the book for them.  
(which results in excerpts with similar meaning but very different wordings) 

### 1. Search for excerpts from a given input

Prepare book and convert every word to its dense vector representation.

In [ ]:
if 'book.txt' not in listdir(join(HOME, 'data')):
    URL = "https://www.gutenberg.org/cache/epub/42671/pg42671.txt"
    r = str(requests.get(URL).content, encoding='utf-8')
    with open(join('data', 'book.txt'), 'w') as file:
        file.write(r)

with open(join('data', 'book.txt'), 'r') as file:
    book = file.read()

#### 1.1. Searching for the original sentence, just to get a feel for the technique and make sure it works.

In [ ]:
excerpt = ("It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.")
locate_excerpt(excerpt, book)

['[324] it is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.']

#### 1.2. Make it a little more difficult and change some of the words for synonims.

Outcome  

```
We can see that there is a slight misalignment in the result: the first 4 words are missing, and 3 additional words (or 4 tokens, considering the dot) are added to the end.  
This is a frequent artifact of the response.  
Since we are working with sliding windows, there is a large overlap between sentences,  
and results will be often ofset by a few words to the right or left.
```

In [ ]:
excerpt = ("It is a fact universally known, that an unmarried man in possession of a vast fortune, must be in need of a wife")
locate_excerpt(excerpt, book)

['[328] universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. however little known']

#### 1.3. Going beyond synonims and alter the sentence more substantially, including some simplifications
- Because we are providing an excerpt that is somewhat shorter than the original, adding a margin to increase the length of the sliding window will help in locating the result.

In [ ]:
excerpt = ("It is a fact universally known, that a man who is rich and single surely wants a wife")
locate_excerpt(excerpt, book, margin = 10)

['[324] it is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. however little known the feelings or views of such a man may be']

#### 1.4. Try an extreme example and provide just the gist of the sentence as an excerpt for the algorithm

Outcome  

```
That did not go very well. 
The algorithm found a match that does not correspond to the original sentence we wanted in the book.
```

In [ ]:
excerpt = "Everyone knows that a rich single man wants a wife"
locate_excerpt(excerpt, book, margin = 10)

['[83860] as he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i']

#### 1.5. Cycle through the top results and see if the actual sentence pops up:

Outcome  

```
Looking carefully at the top 20 results, we see that the original excerpt we were looking for corresponds to entries 5, 6, 15, and 17. There is a lot of overlap between results, so there are actually just 7 different parts in this selection, of which our desired outcome is the third.


In [ ]:
# define excerpt
excerpt = "Everyone knows that a rich single man wants a wife"
margin = 10

# find top 20 results and print them
excerpts = locate_excerpt(excerpt, book, margin = 10, count = 20)
print("\n".join(excerpts))

[83860] as he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i
[111794] is such a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that i
[111793] he is such a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that
[83861] he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i would
[330] , that a single man in possession of a good fortune, must be in want of a wife. however little known the feelings or views of such a
[329] acknowledged, that a single man in possession of a good fortune, must be in want of a wife. however little known the feelings or views of such
[111796] a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that i want very
[111797] man ! '' `` yes, yes, they must marry. there is nothi

#### 1.6. Identify the overlaps and merges them into single results.  
In that case, our desired excerpt would be the third result on the list.

In [ ]:
# Merge overlapping results
merged_results = merge_overlapping_results(excerpts)

# Print merged results
for idx, cluster in enumerate(merged_results, start=1):
    print(f"Cluster {idx}:")
    print("\n".join(cluster))
    print("="*50)

Cluster 1:
[83860] as he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i
[83861] he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i would
[83862] chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and if i was her, i would not
[83854] , well ! it is just as he chooses. nobody wants him to come. though i shall always say that he used my daughter extremely ill ; and
Cluster 2:
[111794] is such a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that i
[111793] he is such a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that
[111796] a man ! '' `` yes, yes, they must marry. there is nothing else to be done. but there are two things that i want very
[111797] man ! '' `` yes, yes, they must marry. 

#### In an actual application, 
it might be necessary for the user to try a couple of different values for the `margin` parameter  
or to scroll through a short list of candidate results.

---
### 2. Search for original text from translations

Instead of searching for sentences with the same meaning,  
we will look for excerpts corresponding to the Brazilian Portuguese translation.

For each of these excerpts, 
1. use the `googletrans` library to translate them into English.  
2. use `locate_excerpt` to try and find the corresponding excerpt in the English book. 

Things to take note
- You will notice that the translation is rather different from the original text. 
- Although it certainly holds the same meaning, the choice and order of words is remarkably different.

#### 2.1. Test single sentences of translated excerpts

In [ ]:
import googletrans
from textwrap import wrap

list_excerpts_translated = ["É uma verdade universalmente conhecida que um homem solteiro, possuidor de uma boa fortuna, deve estar necessitado de esposa."
                      , "Mas quando Elizabeth contou que ele ficara em silêncio, a hipótese não pareceu muito plausível, mesmo para Charlotte que a desejava."
                      , "Mrs. Gardiner ficou surpreendida e preocupada. Mas como se aproximavam agora do lugar onde residira na sua mocidade, ela se entregou toda ao encanto das suas recordações"
                      , "— Acha que eles estão em Londres? — Sim, em que outro lugar poderiam se esconder?"]

for e in list_excerpts_translated:
    excerpt_english = googletrans.Translator().translate(e).text
    result = locate_excerpt(excerpt_english, book)

    print("Excerpt (pt):")
    [print(st) for st in wrap(e)]
    print("\nTranslation (en):")
    [print(st) for st in wrap(excerpt_english)]
    print("\nOriginal (en):")
    [print(st) for st in result]
    print("="*50)

Excerpt (pt):
É uma verdade universalmente conhecida que um homem solteiro,
possuidor de uma boa fortuna, deve estar necessitado de esposa.

Translation (en):
It is a universally known truth that a single man, possessed of a good
fortune, must be in need of a wife.

Original (en):
[324] it is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife
Excerpt (pt):
Mas quando Elizabeth contou que ele ficara em silêncio, a hipótese não
pareceu muito plausível, mesmo para Charlotte que a desejava.

Translation (en):
But when Elizabeth told him that he had been silent, the hypothesis
did not seem very plausible, even to Charlotte who wanted her.

Original (en):
[66371] elizabeth told of his silence, it did not seem very likely, even to charlotte 's wishes, to be the case ; and after
Excerpt (pt):
Mrs. Gardiner ficou surpreendida e preocupada. Mas como se aproximavam
agora do lugar onde residira na sua mocidade, ela se entregou toda ao
enc

#### 2.2. Try a longer excerpt with several sentences:

In [ ]:
excerpt_translated = ("Se pudéssemos saber quais eram as dívidas de Wickham... E com quanto ele dotou nossa irmã... Saberia exatamente o que Mr. Gardiner fez, pois Wickham não tem um tostão de seu. A bondade dos nossos tios é uma coisa que nunca poderá ser paga. Eles a levaram para casa e lhe deram toda a sua proteção e apoio moral. Isto é um sacrifício que anos de gratidão não podem compensar. Nesse momento, ela está em casa deles. Se uma tão grande bondade não lhe der a consciência da falta que praticou, é que ela não merece nunca ser feliz. Imagina a sua cara quando chegar diante da minha tia")

excerpt_english = googletrans.Translator().translate(excerpt_translated).text
result = locate_excerpt(excerpt_english, book)

print("Excerpt (pt):")
[print(st) for st in wrap(excerpt_translated)]
print("\nTranslation (en):")
[print(st) for st in wrap(excerpt_english)]
print("\nOriginal (en):")
[print(st) for st in result]

Excerpt (pt):
Se pudéssemos saber quais eram as dívidas de Wickham... E com quanto
ele dotou nossa irmã... Saberia exatamente o que Mr. Gardiner fez,
pois Wickham não tem um tostão de seu. A bondade dos nossos tios é uma
coisa que nunca poderá ser paga. Eles a levaram para casa e lhe deram
toda a sua proteção e apoio moral. Isto é um sacrifício que anos de
gratidão não podem compensar. Nesse momento, ela está em casa deles.
Se uma tão grande bondade não lhe der a consciência da falta que
praticou, é que ela não merece nunca ser feliz. Imagina a sua cara
quando chegar diante da minha tia

Translation (en):
If we could only know what Wickham's debts were... And how much he
endowed our sister with... We would know exactly what Mr. Gardiner
did, for Wickham has not a penny of his. The kindness of our uncles is
something that can never be repaid. They took her home and gave her
all their protection and moral support. This is a sacrifice that years
of gratitude cannot make up for. Right now,

[None]